In [31]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import ast
from scipy.stats import ttest_ind
import seaborn as sns
import time
import networkx as nx
import warnings


# SettingWithCopyWarning 무시
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# 모든 출력을 보여주도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


In [32]:
parquet_file = 'C:/Users/jjccyy9741/OneDrive - postech.ac.kr/바탕 화면/LG/데이터/postech_company_project.parquet'
# Parquet 파일 열기
table = pq.read_table(parquet_file)

# Table에서 데이터를 DataFrame으로 변환
data = table.to_pandas()
print(data.shape)

(13881284, 71)


In [33]:
df = data.copy()
df = df[['PREV_RECEIPT_NUMBER','RECEIPT_NUMBER']]
print(df.shape)

(13881284, 2)


In [34]:
df11 = df.copy()

In [35]:
n = len(data)

df12 = df11.head(n)

# 시작 시간 기록
start_time = time.time()

print('데이터 크기(n):', n)
print(' ')

# 네트워크 그래프 생성
G = nx.Graph()
for _, row in df12.iterrows():
    if row['PREV_RECEIPT_NUMBER'] is not None:
        G.add_edge(row['PREV_RECEIPT_NUMBER'], row['RECEIPT_NUMBER'])

# 같은 그룹 찾기
connected_components = list(nx.connected_components(G))

# 결과 출력
result_df12 = pd.DataFrame()
if not connected_components:
    # 시퀀스가 하나도 없는 경우
    result_df12 = df12.copy()
    result_df12['group'] = 0
else:
    for idx, component in enumerate(connected_components):
        matching_group = df12[df12['RECEIPT_NUMBER'].isin(component)].copy()  # .copy() 추가
        matching_group['group'] = idx + 1
        result_df12 = pd.concat([result_df12, matching_group], ignore_index=True)

    # 'group' 컬럼으로 정렬
    result_df12 = result_df12.sort_values(by='group')

# 종료 시간 기록
end_time = time.time()
print('실행 시간:', end_time - start_time)
print(' ')

# 빈도가 2 이상인 값을 출력
seq_group = result_df12['group'].value_counts()[result_df12['group'].value_counts() >= 2].index.tolist()
print('시퀀스 그룹 개수:', len(seq_group))
# print('시퀀스 그룹:',seq_group)
print(' ')
print('데이터 크기:', len(result_df12))
display(result_df12.head(10))
# 'group' 열 값의 빈도 계산 및 정렬
group_counts = result_df12['group'].value_counts().sort_values(ascending=False)

# 빈도가 2 이상인 값들만 필터링
result = group_counts[group_counts >= 2]

# 결과 출력
print(result)


데이터 크기(n): 1000
 
실행 시간: 0.35481929779052734
 
시퀀스 그룹 개수: 4
 
데이터 크기: 138


,PREV_RECEIPT_NUMBER,RECEIPT_NUMBER,group
0,LGE_REC_14193717,LGE_REC_0,1
1,LGE_REC_14193718,LGE_REC_1,2
2,LGE_REC_14193719,LGE_REC_2,3
3,LGE_REC_14193720,LGE_REC_3,4
4,LGE_REC_6091,LGE_REC_4,5
5,LGE_REC_14193721,LGE_REC_5,6
6,LGE_REC_14193722,LGE_REC_6,7
7,LGE_REC_14193723,LGE_REC_7,8
8,LGE_REC_14193724,LGE_REC_8,9
9,LGE_REC_14193725,LGE_REC_9,10


group
18    2
64    2
93    2
66    2
Name: count, dtype: int64


In [36]:
result_df12[result_df12['group']==8741]

,PREV_RECEIPT_NUMBER,RECEIPT_NUMBER,group


In [37]:
result_df2 = result_df12.groupby('group').last().reset_index()
print(result_df2.shape)
print(result_df2['group'].nunique())
result_df2.head(10)

(134, 3)
134


,group,PREV_RECEIPT_NUMBER,RECEIPT_NUMBER
0,1,LGE_REC_14193717,LGE_REC_0
1,2,LGE_REC_14193718,LGE_REC_1
2,3,LGE_REC_14193719,LGE_REC_2
3,4,LGE_REC_14193720,LGE_REC_3
4,5,LGE_REC_6091,LGE_REC_4
5,6,LGE_REC_14193721,LGE_REC_5
6,7,LGE_REC_14193722,LGE_REC_6
7,8,LGE_REC_14193723,LGE_REC_7
8,9,LGE_REC_14193724,LGE_REC_8
9,10,LGE_REC_14193725,LGE_REC_9


In [38]:
# result_df2에 있는 RECEIPT_NUMBER 값만 갖는 데이터프레임 생성
data2 = data[data['RECEIPT_NUMBER'].isin(result_df2['RECEIPT_NUMBER'])]
data2.shape

(134, 71)

In [39]:
data2.to_csv('C:/Users/jjccyy9741/OneDrive - postech.ac.kr/바탕 화면/1214_result.csv')